In [58]:
import database_handler
db_session = database_handler.create_connection(None)


In [19]:
import data_handler 
import pandas as pd
import psycopg2

pd.read_csv("C:\\Users\\User\\Downloads\\Fifa_world_cup_matches.csv", encoding= 'utf-8')



,team1,team2,possession team1,possession team2,possession in contest,number of goals team1,number of goals team2,date,hour,category,...,penalties scored team1,penalties scored team2,goal preventions team1,goal preventions team2,own goals team1,own goals team2,forced turnovers team1,forced turnovers team2,defensive pressures applied team1,defensive pressures applied team2
0,QATAR,ECUADOR,42%,50%,8%,0,2,20-Nov-22,17:00,Group A,...,0,1,6,5,0,0,52,72,256,279
1,ENGLAND,IRAN,72%,19%,9%,6,2,21-Nov-22,14:00,Group B,...,0,1,8,13,0,0,63,72,139,416
2,SENEGAL,NETHERLANDS,44%,45%,11%,0,2,21-Nov-22,17:00,Group A,...,0,0,9,15,0,0,63,73,263,251
3,UNITED STATES,WALES,51%,39%,10%,1,1,21-Nov-22,20:00,Group B,...,0,1,7,7,0,0,81,72,242,292
4,ARGENTINA,SAUDI ARABIA,64%,24%,12%,1,2,22-Nov-22,11:00,Group C,...,1,0,4,14,0,0,65,80,163,361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,ENGLAND,FRANCE,54%,36%,10%,1,2,10-Dec-22,20:00,Quarter-final,...,1,0,9,15,0,0,49,54,193,308
60,ARGENTINA,CROATIA,34%,54%,12%,3,0,13-Dec-22,20:00,Semi-final,...,1,0,12,10,0,0,85,63,321,260
61,FRANCE,MOROCCO,34%,55%,11%,2,0,14-Dec-22,20:00,Semi-final,...,0,0,13,14,0,0,72,47,328,218
62,CROATIA,MOROCCO,45%,45%,10%,2,1,17-Dec-22,16:00,Play-off for third place,...,0,0,7,14,0,0,75,72,288,277


In [27]:
import prehook
prehook.create_staging_tables(db_session, "C:\\Users\\User\\Downloads\\Fifa_world_cup_matches.csv",encoding= 'latin1')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (prehook.py, line 25)

In [66]:
from data_handler import create_staging_table 

db_name = 'MarsProjet'
user = 'postgres'
password = 'sql22'
host = 'localhost'
port = '5432'
table_name = 'allplayers_staging'
column_names = ['Coach_name TEXT', 'Country_code TEXT', 'Player_name TEXT', 'Players_id INT']

create_staging_table(db_name, user, password, host, port, table_name, column_names)


UnboundLocalError: local variable 'cur' referenced before assignment

In [67]:
import data_handler 
import pandas as pd
import psycopg2

pd.read_csv("C:\\Users\\User\\Desktop\\excell_fouad\\Groups.csv", encoding='latin1')
conn = psycopg2.connect(
    dbname='MarsProjet',
    user='postgres',
    password='sql22',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

# Create a staging table in the database
cur.execute('''
    CREATE TABLE Groups_staging (
       Year INT,
       Stage TEXT,
       City TEXT,
       Home_team_name TEXT,
       Away_team_name TEXT,
       Atendance INT,
       Hometeam_code TEXT,        
       Awayteam_code TEXT, 
       Match_id INT
    );
''')


conn.commit()
cur.close()
conn.close()



In [37]:
import data_handler 
import pandas as pd
import psycopg2

pd.read_csv("C:\\Users\\User\\Desktop\\excell_fouad\\Teams.csv", encoding='latin1')

conn = psycopg2.connect(
    dbname='MarsProjet',
    user='postgres',
    password='sql22',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

# Create a staging table in the database
cur.execute('''
    CREATE TABLE Teams_staging (
       Teams TEXT,
       Country_code TEXT,
       id INT
    );
''')


conn.commit()
cur.close()
conn.close()




In [45]:
import pandas as pd
import psycopg2

from datetime import datetime


conn = psycopg2.connect(
    dbname='MarsProjet',
    user='postgres',
    password='sql22',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

cur.execute("SELECT last_processed_date FROM etl_watermark WHERE id = 1;")
last_processed_date = cur.fetchone()

if last_processed_date:
    last_processed_date = last_processed_date[0]
else:
    last_processed_date = datetime(2023, 3, 4)  


data = pd.read_csv("C:\\Users\\User\\Desktop\\excell_fouad\\Winners.csv", encoding='latin1')


filtered_data = data[data['date'] > last_processed_date]



conn.commit()
cur.close()
conn.close()


KeyError: 'date'

In [32]:
import pandas as pd
import psycopg2

# Read CSV data into pandas DataFrame
data=pd.read_csv("C:\\Users\\User\\Desktop\\excell_fouad\\Winners.csv", encoding='latin1')

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='MarsProjet',
    user='postgres',
    password='sql22',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

# Create a staging table in the database
cur.execute('''
    CREATE TABLE winners_staging (
        Year INT,
        Country TEXT,
        Country_code TEXT,
        Team_id INT,
        GoalsScored INT,
        QualifiedTeams INT,
        MatchesPlayed INT,
        Attendance INT
    );
''')

# Insert data from pandas DataFrame into the staging table
for index, row in data.iterrows():
    cur.execute('''
        INSERT INTO winners_staging (Year, Country, Country_code, Team_id, GoalsScored, QualifiedTeams, MatchesPlayed, Attendance)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    ''', (row['Year'], row['Country'], row['Country_code'], row['Team_id'], row['GoalsScored'], row['QualifiedTeams'], row['MatchesPlayed'], row['Attendance']))

# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()


DuplicateTable: relation "winners_staging" already exists


In [18]:
import data_handler

film_dataframe = data_handler.returns_query_as_dataframe(db_session, 'SELECT * FROM MarsProjet.equipes')


NameError: name 'db_session' is not defined

In [ ]:
import lookups
import pandas_handler
def return_dataframe_info(dataframe):
    info_dictionary = dict()
    info_dictionary['dataframe_length'] = len(dataframe)
    info_dictionary['total_count_of_table_id'] = int(dataframe.describe().iloc[0][0])
    return info_dictionary

def return_dataframe_subset_from_to_location(dataframe, from_loc, to_loc):
    return dataframe.loc[from_loc,to_loc]
def return_filtered_dataframe(dataframe, filter_field, filter_value):
    return dataframe[dataframe[filter_field.value] > filter_value]

: 

In [ ]:
import database_handler
database_handler.create_connection(None)
    
